In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    TimestampType,
    StringType,
)
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/19 13:13:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/19 13:13:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/10/19 13:13:44 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to spark-events/eventlog_v2_app-20251019131339-0012/events_1_app-20251019131339-0012. This is unsupported


In [3]:
spark

In [4]:
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------+----------------+
|catalog|       namespace|
+-------+----------------+
|   demo|retail_analytics|
+-------+----------------+



In [5]:
spark.sql("SHOW TABLES IN demo.retail_analytics").show()

+----------------+------------------+-----------+
|       namespace|         tableName|isTemporary|
+----------------+------------------+-----------+
|retail_analytics|  fact_order_items|      false|
|retail_analytics|      dim_customer|      false|
|retail_analytics|          dim_date|      false|
|retail_analytics|fact_order_summary|      false|
|retail_analytics|       dim_product|      false|
|retail_analytics|        dim_seller|      false|
+----------------+------------------+-----------+



In [6]:
spark.sql("SELECT * FROM dim_date order by date_key limit 5").show()

+--------+----------+----+-----+----+-----------+------------+-------+-----------+------------+--------------+----------+
|date_key|      date|year|month| day|day_of_week|week_of_year|quarter|day_of_year|is_month_end|is_month_start|is_weekend|
+--------+----------+----+-----+----+-----------+------------+-------+-----------+------------+--------------+----------+
|       0|      NULL|NULL| NULL|NULL|       NULL|        NULL|   NULL|       NULL|        NULL|          NULL|      NULL|
|       1|2020-01-01|2020|    1|   1|          4|           1|      1|          1|       false|          true|     false|
|       2|2020-01-02|2020|    1|   2|          5|           1|      1|          2|       false|         false|     false|
|       3|2020-01-03|2020|    1|   3|          6|           1|      1|          3|       false|         false|     false|
|       4|2020-01-04|2020|    1|   4|          7|           1|      1|          4|       false|         false|      true|
+--------+----------+---

In [8]:
spark.sql("SELECT * FROM dim_seller order by seller_id limit 5").show()

+--------------------+---------+--------------------+-----+---------------+--------------+------------+----------+
|           seller_sk|seller_id|             address|state|zip_code_prefix|effective_from|effective_to|is_current|
+--------------------+---------+--------------------+-----+---------------+--------------+------------+----------+
|e38b2c60698825512...| 00000001|2323 Johnson Lodg...|   GA|          30836|    2020-08-01|        NULL|      true|
|59166d10a430ac653...| 00000002|125 Bradley Oval ...|   PA|          16524|    2021-04-04|        NULL|      true|
|b8fb70c9dd81e4f39...| 00000003|26992 Patrick Loc...|   MT|          59418|    2020-02-13|        NULL|      true|
|46272636b85200be0...| 00000004|56999 Brianna Street|   OH|          45471|    2020-09-30|        NULL|      true|
|25fb7dc683d837086...| 00000005|58768 Mckinney Mi...|   HI|          96857|    2021-01-29|        NULL|      true|
+--------------------+---------+--------------------+-----+---------------+-----

In [9]:
spark.sql("SELECT * FROM dim_customer order by customer_id limit 5").show()

+--------------------+-----------+--------------------+-----+---------------+--------------+------------+----------+
|         customer_sk|customer_id|             address|state|zip_code_prefix|effective_from|effective_to|is_current|
+--------------------+-----------+--------------------+-----+---------------+--------------+------------+----------+
|846bde2ad1497770b...|   00000001|452 Jones Village...|   SC|          29763|    2024-04-19|        NULL|      true|
|bbcff9338aabe3807...|   00000002|80988 Santiago Lo...|   DE|          19860|    2022-01-19|        NULL|      true|
|4360c04c0747663fa...|   00000003|98327 Melanie Key...|   AK|          99616|    2021-09-07|        NULL|      true|
|6b8f776fc6eb9617f...|   00000004|998 Chelsea Shoal...|   WA|          98633|    2024-09-24|        NULL|      true|
|4361a3634ac6ee7bd...|   00000005|0229 Jones Street...|   KY|          41543|    2022-10-01|        NULL|      true|
+--------------------+-----------+--------------------+-----+---

In [10]:
spark.sql("SELECT * FROM dim_product order by product_id limit 5").show()

+--------------------+----------+--------------+---------------+------+-----------+---------+---------+--------+--------------+------------+----------+
|          product_sk|product_id|  product_name|  category_name| price| size_label|length_cm|height_cm|width_cm|effective_from|effective_to|is_current|
+--------------------+----------+--------------+---------------+------+-----------+---------+---------+--------+--------------+------------+----------+
|7e5fcfca2a05c1605...|  00000001|Boxify Classic|general_purpose| 35.20|      Small|     25.5|     15.5|    20.5|    2020-01-01|        NULL|      true|
|35a15483c3f309228...|  00000002|Boxify Classic|general_purpose| 60.50|     Medium|     35.5|     20.5|    30.5|    2020-01-01|        NULL|      true|
|cd760fd4592c84bd2...|  00000003|Boxify Classic|general_purpose| 85.80|      Large|     50.5|     30.5|    40.5|    2020-01-01|        NULL|      true|
|c809e19607a297b74...|  00000004|Boxify Classic|general_purpose|101.20|Extra Large|     

In [19]:
spark.sql("SELECT * FROM fact_order_items order by order_id limit 5").show()

+--------------------+--------------------+--------+-------------+----------+-------------+--------------+-----------------------+
|           seller_sk|          product_sk|order_id|order_item_id|item_value|freight_value|order_date_key|shipping_limit_date_key|
+--------------------+--------------------+--------+-------------+----------+-------------+--------------+-----------------------+
|098fcfae4616f1418...|455ff9fe598546e1f...|00000001|            0|    128.80|        30.64|          1356|                   1357|
|098fcfae4616f1418...|721d41d23b238229e...|00000001|            1|     38.40|         3.54|          1356|                   1357|
|098fcfae4616f1418...|7d70a0c44e3a4a783...|00000001|            3|    138.00|        45.00|          1356|                   1357|
|098fcfae4616f1418...|5f79c10f14b419e6f...|00000001|            2|     48.00|        13.77|          1356|                   1357|
|8d6493b74a71bc6d6...|fa0a2b69fa1ccdd24...|00000002|            0|     48.00|      

In [18]:
spark.read.parquet("s3a://demo-input/order_items.parquet").filter(F.col("order_id") =='00000001').show()

+--------+-------------+----------+---------+-------------------+-----+-------------+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|
+--------+-------------+----------+---------+-------------------+-----+-------------+
|00000001|            0|  00000088| 00000013|         2023-09-18|128.8|        30.64|
|00000001|            1|  00000057| 00000013|         2023-09-18| 38.4|         3.54|
|00000001|            2|  00000033| 00000013|         2023-09-18| 48.0|        13.77|
|00000001|            3|  00000036| 00000013|         2023-09-18|138.0|         45.0|
+--------+-------------+----------+---------+-------------------+-----+-------------+



In [20]:
spark.sql("SELECT * FROM fact_order_summary order by order_id limit 10").show()

+--------------------+--------+------------+-----------------+-------------------+----------+--------------+-----------------------------+------------------------------+---------------------------+-----------------------+---------------------------+---------------------------+
|         customer_sk|order_id|order_status|total_order_value|total_freight_value|item_count|order_date_key|delivered_to_carrier_date_key|delivered_to_customer_date_key|estimated_delivery_date_key|order_approved_date_key|min_shipping_limit_date_key|max_shipping_limit_date_key|
+--------------------+--------+------------+-----------------+-------------------+----------+--------------+-----------------------------+------------------------------+---------------------------+-----------------------+---------------------------+---------------------------+
|4f143e84c5a08db2e...|00000001|     SHIPPED|           353.20|              92.95|         4|          1356|                            0|                            

In [23]:
spark.read.parquet("s3a://demo-input/orders.parquet").filter(F.col("order_id").isin(['00000001', '00000002'])).show()

+--------+-----------+------------+-------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_date|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+-------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|00000001|   00000598|     SHIPPED|         2023-09-17|       2023-09-18|                        NULL|                         NULL|                   2023-09-21|
|00000002|   00001091|    APPROVED|         2024-11-04|       2024-11-05|                        NULL|                         NULL|                         NULL|
+--------+-----------+------------+-------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



In [25]:
spark.stop()